In [1]:
!nvidia-smi

Tue Dec 14 19:37:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install simpletransformers
!pip install tensorboardx

     |████████████████████████████████| 247 kB 5.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 3.3 MB 30.2 MB/s 
     |████████████████████████████████| 1.7 MB 37.9 MB/s 
     |████████████████████████████████| 9.1 MB 30.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.3 MB/s 
     |████████████████████████████████| 298 kB 47.5 MB/s 
     |████████████████████████████████| 3.3 MB 35.4 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 596 kB 37.8 MB/s 
     |████████████████████████████████| 61 kB 458 kB/s 
     |████████████████████████████████| 140 kB 38.6 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 180 kB 43.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 243 kB 39.3 MB/s 
     |████████████████████████████████| 132 kB 44.3 MB/s 
     |█████████████████

     |████████████████████████████████| 124 kB 5.4 MB/s 


In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
import sklearn

In [2]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [3]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


2021-12-14 19:39:17.170323: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


In [4]:
from pathlib import Path
import re

def read_dataset(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = raw_text.split('\n')[4:]
#     ids_docs = []
#     keywords_docs = []
#     texts_docs = []
#     labels_docs = []
    ids = []
    keywords = []
    texts = []
    labels = []
    for line in raw_docs:
        
#         for line in doc.split('\n'):
        id_, keyword, text, label = line.split('\t')[0],line.split('\t')[2],line.split('\t')[4],line.split('\t')[5]
        ids.append(id_)
        keywords.append(keyword)
        texts.append(text)
        if(label=='0' or label=='1'):
            labels.append(0)
        else:
            labels.append(1)
#         labels.append(label)
#         token_docs.append(tokens)
#         tag_docs.append(tags)

    return ids, keywords, texts, labels

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
IDs,Keywords,Texts,Labels = read_dataset('/content/gdrive/My Drive/Colab Notebooks/dontpatronizeme_pcl.tsv')

In [7]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(Texts, Labels, test_size=.2)

In [8]:
train_dataset = []
for i in range(len(train_texts)):
  train_dataset.append({'text':train_texts[i],'label':train_tags[i]})

In [9]:
train_df = pd.DataFrame(train_dataset)

In [10]:
val_dataset = []
for i in range(len(val_texts)):
  val_dataset.append({'text':val_texts[i],'label':val_tags[i]})

In [11]:
val_df = pd.DataFrame(val_dataset)

In [33]:
total_no_1s = len(train_df[train_df.label==1])

In [37]:
training_set = pd.concat([train_df[train_df.label==1],train_df[train_df.label==0][3500:3500+total_no_1s*2]])

In [38]:
total_no_1s

787

In [39]:
len(training_set)

2361

In [40]:
model_args = ClassificationArgs(num_train_epochs=10, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
model.train_model(training_set[['text', 'label']])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/2361 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/296 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/296 [00:00<?, ?it/s]

...

Running Epoch 7 of 10:   0%|          | 0/296 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/296 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/296 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(2960, 0.3377388615060497)

In [41]:
val_total_no_1s = len(val_df[val_df.label==1])

In [42]:
val_total_no_1s

206

In [43]:
testing_set = pd.concat([val_df[val_df.label==1],val_df[val_df.label==0][:val_total_no_1s*2]])

In [44]:
testing_set.text

12     In his view , Professor George Gongera , an ex...
26     Remarkable too are the ways in which a childho...
37     """ Mike 's leadership has made NSW the econom...
41     Wow , all five of these women are so dedicated...
42     """ There is no one who can tell immigrants th...
                             ...                        
447    There are many philosophical and ethical facto...
448    We see the lack of substantial networks of sup...
449    Swabhoomi addresses the issues migrants faced ...
450    On July 17 , 2015 , Opposition Senator Robert ...
451    Around 95% of houses were destroyed , leaving ...
Name: text, Length: 618, dtype: object

In [45]:
predictions, _ = model.predict(testing_set.text.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/618 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

In [46]:
len(predictions),len(testing_set.label)

(618, 618)

In [48]:
sklearn.metrics.precision_recall_fscore_support(testing_set.label,predictions,average=None)

(array([0.85371703, 0.72139303]),
 array([0.86407767, 0.7038835 ]),
 array([0.8588661 , 0.71253071]),
 array([412, 206]))

In [49]:
sklearn.metrics.precision_recall_fscore_support(testing_set.label,predictions,average='binary')

(0.7213930348258707, 0.7038834951456311, 0.7125307125307127, None)